# Problem Statement
The objective of this project is to develop a machine learning model that can accurately detect and classify diseases in rice plant leaves. Rice is one of the most important staple crops globally, and diseases can significantly impact its yield and quality. Early and accurate detection of diseases in rice plants can enable timely intervention and effective disease management strategies.

### Key challenges:

Multi-class Classification: The model should be able to classify rice leaf images into multiple disease categories, such as Brown Spot, Leaf Smut, and Bacterial Leaf Blight. Each disease has distinct visual symptoms and patterns that need to be learned by the model.

Real-time Detection: The model should be capable of real-time disease detection in rice plant leaves. It should be able to process images quickly and provide accurate predictions, allowing farmers or agricultural experts to take prompt actions based on the results.

Generalization to New Samples: The trained model should have good generalization capabilities and be able to detect diseases accurately on unseen images. It should be robust to variations in lighting conditions, image quality, and different rice leaf varieties.

### Solution

Preprocess the dataset by resizing, normalizing, and augmenting the images to enhance the model's ability to learn and generalize.

Model Selection and Architecture Design: Choose an appropriate pre-trained model or design a custom architecture suitable for the rice leaf disease detection task. The model should have sufficient capacity to capture the complex patterns associated with different diseases.

Transfer Learning and Fine-tuning: Utilize transfer learning by leveraging pre-trained models trained on large-scale image datasets. Fine-tune the selected model on the rice leaf disease dataset to adapt it to the specific task and disease classes.

Model Training and Evaluation: Split the dataset into training and validation sets. Train the model using the training set and optimize its performance using appropriate loss functions, optimizers, and learning rate schedules. Evaluate the model's performance on the validation set using appropriate evaluation metrics such as accuracy, precision, recall, and F1-score.

Hyperparameter Tuning: Fine-tune hyperparameters such as learning rate, batch size, regularization techniques, and model architecture to optimize the model's performance and improve its accuracy.

Real-time Deployment: Once the model achieves satisfactory performance on the validation set, deploy it to a real-time environment where it can process incoming rice leaf images and provide disease predictions in real-time.
### Target 
By developing an accurate and efficient rice leaf disease detection model, this project aims to assist farmers and agricultural experts in detecting diseases early, implementing timely interventions, and effectively managing rice plant health, leading to improved crop yields and reduced economic losses.

In [1]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

In [49]:
# Step 1: Data Preparation
data_dir = r'C:\\Users\\bdiko\\Downloads\\PRCP-1001-RiceLeaf-10\\Data\\'
class_names = ['Bacterial leaf blight', 'Brown spot', 'Leaf smut']
image_size = (224, 224)  # Adjust the size according to your requirements


In [50]:
data_dir = 'C:\\Users\\bdiko\\Downloads\\PRCP-1001-RiceLeaf-10\\Data\\'
print(os.listdir(data_dir))

['Bacterial leaf blight', 'Brown spot', 'Leaf smut']


In [54]:
# Step 2: Data Preprocessing
def preprocess_images(image_paths):
    images = []
    for path in image_paths:
        img = Image.open(path)
        img = img.resize(image_size)
        img = np.array(img) / 255.0  # Normalize pixel values
        images.append(img)
    return np.array(images)

X = []
y = []

for class_name in class_names:
    class_dir = os.path.join(data_dir, class_name)
    image_paths = [os.path.join(class_dir, img) for img in os.listdir(class_dir) if img.endswith('.jpg')]
    labels = [class_names.index(class_name)] * len(image_paths)
    X.extend(image_paths)
    y.extend(labels)
    
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = preprocess_images(X_train)
X_test = preprocess_images(X_test)


In [64]:
# Convert y_train and y_test to numpy arrays
y_train = np.array(y_train)
y_test = np.array(y_test)

In [65]:
# Step 3: Model Development
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(image_size[0], image_size[1], 3))
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(class_names), activation='softmax'))

In [66]:
# Step 4: Model Training
batch_size = 32
epochs = 10

optimizer = Adam(lr=0.001)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

train_datagen = ImageDataGenerator(rotation_range=20, width_shift_range=0.2, height_shift_range=0.2,
                                   horizontal_flip=True)
train_generator = train_datagen.flow(X_train, y_train, batch_size=batch_size)

model.fit(train_generator, epochs=epochs, validation_data=(X_test, y_test))


Epoch 1/10
2/2 [==============================] - 19s 5s/step - loss: 10.2007 - accuracy: 0.4878 - val_loss: 1.0885 - val_accuracy: 0.3636
Epoch 2/10
2/2 [==============================] - 18s 5s/step - loss: 0.9998 - accuracy: 0.5366 - val_loss: 1.9661 - val_accuracy: 0.4545
Epoch 3/10
2/2 [==============================] - 18s 14s/step - loss: 1.6331 - accuracy: 0.4878 - val_loss: 1.0555 - val_accuracy: 0.3636
Epoch 4/10
2/2 [==============================] - 18s 5s/step - loss: 1.1630 - accuracy: 0.2927 - val_loss: 1.0616 - val_accuracy: 0.4545
Epoch 5/10
2/2 [==============================] - 18s 5s/step - loss: 0.8914 - accuracy: 0.6098 - val_loss: 1.1523 - val_accuracy: 0.4545
Epoch 6/10
2/2 [==============================] - 18s 5s/step - loss: 1.0797 - accuracy: 0.5854 - val_loss: 1.1587 - val_accuracy: 0.3636
Epoch 7/10
2/2 [==============================] - 18s 5s/step - loss: 1.1093 - accuracy: 0.5122 - val_loss: 1.1067 - val_accuracy: 0.3636
Epoch 8/10
2/2 [================

In [67]:
# Step 5: Model Evaluation
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")


1/1 [==============================] - 1s 1s/step - loss: 1.0786 - accuracy: 0.3636
Test Loss: 1.0786045789718628
Test Accuracy: 0.3636363744735718


In [70]:
# Step 6: Predictions
test_images = X_test[:5]
predictions = model.predict(test_images)

for i in range(len(test_images)):
    predicted_class_index = np.argmax(predictions[i])
    predicted_class = class_names[predicted_class_index]
    print(f"Image: {test_images[i]} - Predicted Class: {predicted_class}")

1/1 [==============================] - 1s 528ms/step
Image: [[[1.         1.         1.        ]
  [1.         1.         1.        ]
  [1.         1.         1.        ]
  ...
  [0.86666667 0.85490196 0.21176471]
  [0.87058824 0.85882353 0.23529412]
  [0.86666667 0.85490196 0.24705882]]

 [[1.         1.         1.        ]
  [1.         1.         1.        ]
  [1.         1.         1.        ]
  ...
  [0.85490196 0.84313725 0.2       ]
  [0.85882353 0.84705882 0.22352941]
  [0.85490196 0.84313725 0.23529412]]

 [[1.         1.         1.        ]
  [1.         1.         1.        ]
  [1.         1.         1.        ]
  ...
  [0.82745098 0.81568627 0.16470588]
  [0.83137255 0.81960784 0.19215686]
  [0.82745098 0.81568627 0.2       ]]

 ...

 [[0.76862745 0.59607843 0.25490196]
  [0.78039216 0.60392157 0.26666667]
  [0.78823529 0.61568627 0.27058824]
  ...
  [0.4        0.39607843 0.01176471]
  [0.40784314 0.40392157 0.01960784]
  [0.41960784 0.40392157 0.02352941]]

 [[0.74901961 

## Improve model accuracy

In [71]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping


In [76]:
# Step 1: Data Preparation
data_dir = r'C:\\Users\\bdiko\\Downloads\\PRCP-1001-RiceLeaf-10\\Data\\'
class_names = ['Bacterial leaf blight', 'Brown spot', 'Leaf smut']
image_size = (224, 224)  # Adjust the size according to your requirements


In [79]:
# Step 2: Data Preprocessing
def preprocess_images(images):
    processed_images = []
    for img in images:
        img = Image.fromarray((img * 255).astype(np.uint8))
        img = img.resize(image_size)
        img = np.array(img) / 255.0  # Normalize pixel values
        processed_images.append(img)
    return np.array(processed_images)

X = []
y = []

for class_name in class_names:
    class_dir = os.path.join(data_dir, class_name)
    image_paths = [os.path.join(class_dir, img) for img in os.listdir(class_dir) if img.endswith('.jpg')]
    labels = [class_names.index(class_name)] * len(image_paths)
    X.extend(image_paths)
    y.extend(labels)
    
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [81]:
# Convert X_train and X_test to numpy arrays
X_train = preprocess_images(X_train)
X_test = preprocess_images(X_test)

In [82]:
# Convert y_train and y_test to numpy arrays
y_train = np.array(y_train)
y_test = np.array(y_test)


In [83]:
# Step 3: Model Development
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(image_size[0], image_size[1], 3))
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(class_names), activation='softmax'))

In [84]:
# Step 4: Model Training
batch_size = 32
epochs = 50

optimizer = Adam(lr=0.001)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

train_datagen = ImageDataGenerator(rotation_range=20, width_shift_range=0.2, height_shift_range=0.2,
                                   horizontal_flip=True, shear_range=0.2, zoom_range=0.2)
train_generator = train_datagen.flow(X_train, y_train, batch_size=batch_size)

# Early stopping callback to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model.fit(train_generator, epochs=epochs, validation_data=(X_test, y_test), callbacks=[early_stopping])


Epoch 1/50
2/2 [==============================] - 19s 13s/step - loss: 5.5921 - accuracy: 0.4878 - val_loss: 1.1378 - val_accuracy: 0.3636
Epoch 2/50
2/2 [==============================] - 17s 13s/step - loss: 3.1331 - accuracy: 0.4146 - val_loss: 1.0667 - val_accuracy: 0.3636
Epoch 3/50
2/2 [==============================] - 17s 5s/step - loss: 1.0603 - accuracy: 0.4146 - val_loss: 1.0538 - val_accuracy: 0.3636
Epoch 4/50
2/2 [==============================] - 17s 5s/step - loss: 0.8726 - accuracy: 0.5854 - val_loss: 1.2166 - val_accuracy: 0.3636
Epoch 5/50
2/2 [==============================] - 18s 14s/step - loss: 1.2686 - accuracy: 0.3415 - val_loss: 1.1045 - val_accuracy: 0.4545
Epoch 6/50
2/2 [==============================] - 18s 14s/step - loss: 1.1722 - accuracy: 0.2927 - val_loss: 1.1122 - val_accuracy: 0.4545
Epoch 7/50
2/2 [==============================] - 18s 5s/step - loss: 1.2156 - accuracy: 0.3171 - val_loss: 1.0924 - val_accuracy: 0.4545
Epoch 8/50
2/2 [==============

In [85]:
# Step 6: Predictions
test_images = X_test[:5]
preprocessed_test_images = preprocess_images(test_images)
predictions = model.predict(preprocessed_test_images)

for i, image_path in enumerate(test_images):
    predicted_class_index = np.argmax(predictions[i])
    predicted_class = class_names[predicted_class_index]
    print(f"Image: {image_path} - Predicted Class: {predicted_class}")

1/1 [==============================] - 0s 472ms/step
Image: [[[0.00392157 0.00392157 0.00392157]
  [0.00392157 0.00392157 0.00392157]
  [0.00392157 0.00392157 0.00392157]
  ...
  [0.1372549  0.14901961 0.79215686]
  [0.13333333 0.14509804 0.76862745]
  [0.1372549  0.14901961 0.75686275]]

 [[0.00392157 0.00392157 0.00392157]
  [0.00392157 0.00392157 0.00392157]
  [0.00392157 0.00392157 0.00392157]
  ...
  [0.14901961 0.16078431 0.80392157]
  [0.14509804 0.15686275 0.78039216]
  [0.14901961 0.16078431 0.76862745]]

 [[0.00392157 0.00392157 0.00392157]
  [0.00392157 0.00392157 0.00392157]
  [0.00392157 0.00392157 0.00392157]
  ...
  [0.17647059 0.18823529 0.83921569]
  [0.17254902 0.18431373 0.81176471]
  [0.17647059 0.18823529 0.80392157]]

 ...

 [[0.23529412 0.40784314 0.74901961]
  [0.22352941 0.4        0.7372549 ]
  [0.21568627 0.38823529 0.73333333]
  ...
  [0.60392157 0.60784314 1.        ]
  [0.59607843 0.6        1.        ]
  [0.58431373 0.6        1.        ]]

 [[0.25490196 

In [86]:
from sklearn.metrics import classification_report

# Make predictions on the test set
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)

# Generate classification report
report = classification_report(y_test, y_pred, target_names=class_names)
print(report)

1/1 [==============================] - 1s 845ms/step
                       precision    recall  f1-score   support

Bacterial leaf blight       0.00      0.00      0.00         2
           Brown spot       0.36      1.00      0.53         4
            Leaf smut       0.00      0.00      0.00         5

             accuracy                           0.36        11
            macro avg       0.12      0.33      0.18        11
         weighted avg       0.13      0.36      0.19        11



C:\Users\bdiko\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\bdiko\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\bdiko\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [87]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

1/1 [==============================] - 1s 891ms/step - loss: 1.0538 - accuracy: 0.3636
Test Loss: 1.0538
Test Accuracy: 0.3636


### Data augmentation: Apply more aggressive data augmentation techniques to further increase the diversity of your training data. Experiment with a wider range of transformations, such as random rotations, shearing, scaling, and flipping. This can help the model learn more robust features and improve its ability to generalize.

In [90]:
# Step 1: Data Preparation
data_dir =  r'C:\\Users\\bdiko\\Downloads\\PRCP-1001-RiceLeaf-10\\Data\\'
class_names = ['Bacterial leaf blight', 'Brown spot', 'Leaf smut']
image_size = (224, 224)  # Adjust the size according to your requirements


In [91]:
# Step 2: Data Preprocessing
def preprocess_images(images):
    processed_images = []
    for img in images:
        img = Image.fromarray((img * 255).astype(np.uint8))
        img = img.resize(image_size)
        img = np.array(img) / 255.0  # Normalize pixel values
        processed_images.append(img)
    return np.array(processed_images)

X = []
y = []

for class_name in class_names:
    class_dir = os.path.join(data_dir, class_name.lower())  # Convert to lowercase
    image_paths = [os.path.join(class_dir, img) for img in os.listdir(class_dir) if img.endswith('.jpg')]
    labels = [class_names.index(class_name)] * len(image_paths)
    X.extend(image_paths)
    y.extend(labels)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [92]:

# Convert X_train and X_test to numpy arrays
X_train = preprocess_images(np.array([np.array(Image.open(fname)) for fname in X_train]))
X_test = preprocess_images(np.array([np.array(Image.open(fname)) for fname in X_test]))


C:\Users\bdiko\AppData\Local\Temp\ipykernel_9024\1264991613.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_train = preprocess_images(np.array([np.array(Image.open(fname)) for fname in X_train]))
C:\Users\bdiko\AppData\Local\Temp\ipykernel_9024\1264991613.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_test = preprocess_images(np.array([np.array(Image.open(fname)) for fname in X_test]))


In [93]:

# Convert y_train and y_test to numpy arrays
y_train = np.array(y_train)
y_test = np.array(y_test)


In [94]:
# Step 3: Model Development
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(image_size[0], image_size[1], 3))
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(class_names), activation='softmax'))

In [95]:
# Step 4: Model Training
batch_size = 32
epochs = 50

optimizer = Adam(lr=0.001)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

train_datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow(X_train, y_train, batch_size=batch_size)

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model.fit(train_generator, epochs=epochs, validation_data=(X_test, y_test), callbacks=[early_stopping])


Epoch 1/50
2/2 [==============================] - 19s 14s/step - loss: 35.4557 - accuracy: 0.4146 - val_loss: 1.0892 - val_accuracy: 0.3636
Epoch 2/50
2/2 [==============================] - 18s 5s/step - loss: 1.1313 - accuracy: 0.2927 - val_loss: 1.3416 - val_accuracy: 0.4545
Epoch 3/50
2/2 [==============================] - 18s 5s/step - loss: 1.4481 - accuracy: 0.3659 - val_loss: 1.0359 - val_accuracy: 0.4545
Epoch 4/50
2/2 [==============================] - 18s 14s/step - loss: 0.9868 - accuracy: 0.4634 - val_loss: 1.0449 - val_accuracy: 0.4545
Epoch 5/50
2/2 [==============================] - 18s 14s/step - loss: 1.0819 - accuracy: 0.3902 - val_loss: 1.0436 - val_accuracy: 0.3636
Epoch 6/50
2/2 [==============================] - 18s 5s/step - loss: 0.9666 - accuracy: 0.5366 - val_loss: 1.0552 - val_accuracy: 0.3636
Epoch 7/50
2/2 [==============================] - 18s 14s/step - loss: 0.9809 - accuracy: 0.5366 - val_loss: 1.0536 - val_accuracy: 0.3636
Epoch 8/50
2/2 [=============

In [96]:
# Step 5: Evaluation
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

1/1 [==============================] - 1s 1s/step - loss: 1.0359 - accuracy: 0.4545
Test Loss: 1.0359
Test Accuracy: 0.4545


It seems that the accuracy of the model on the test set is 45.45%. This result indicates that the model is not performing well and may require further improvements.

## Adjust hyperparameters: Experiment with different hyperparameters, such as learning rate, batch size, and optimizer. You can try decreasing the learning rate, increasing the batch size, or using a different optimizer to see if it improves the model's performance


In [97]:
# Step 4: Model Training
batch_size = 64
epochs = 50

optimizer = Adam(lr=0.0001)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

train_datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow(X_train, y_train, batch_size=batch_size)

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model.fit(train_generator, epochs=epochs, validation_data=(X_test, y_test), callbacks=[early_stopping])


Epoch 1/50
1/1 [==============================] - 19s 19s/step - loss: 1.0550 - accuracy: 0.3415 - val_loss: 1.1165 - val_accuracy: 0.3636
Epoch 2/50
1/1 [==============================] - 17s 17s/step - loss: 0.9786 - accuracy: 0.4878 - val_loss: 9.0743 - val_accuracy: 0.4545
Epoch 3/50
1/1 [==============================] - 18s 18s/step - loss: 9.6802 - accuracy: 0.4390 - val_loss: 1.0555 - val_accuracy: 0.3636
Epoch 4/50
1/1 [==============================] - 17s 17s/step - loss: 1.0435 - accuracy: 0.3171 - val_loss: 1.0560 - val_accuracy: 0.3636
Epoch 5/50
1/1 [==============================] - 18s 18s/step - loss: 0.9394 - accuracy: 0.4878 - val_loss: 1.0484 - val_accuracy: 0.3636
Epoch 6/50
1/1 [==============================] - 18s 18s/step - loss: 0.9849 - accuracy: 0.5366 - val_loss: 1.0731 - val_accuracy: 0.3636
Epoch 7/50
1/1 [==============================] - 19s 19s/step - loss: 1.0016 - accuracy: 0.4390 - val_loss: 1.0354 - val_accuracy: 0.3636
Epoch 8/50
1/1 [===========

In [98]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

1/1 [==============================] - 1s 924ms/step - loss: 1.0354 - accuracy: 0.3636
Test Loss: 1.0354
Test Accuracy: 0.3636


## Ensemble learning involves combining multiple models to improve performance. 

In [99]:
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [100]:
# Define the base models
model1 = SVC(kernel='rbf', probability=True)
model2 = RandomForestClassifier(n_estimators=100)
model3 = LogisticRegression()

In [101]:
# Create the ensemble model
ensemble_model = VotingClassifier(estimators=[('svm', model1), ('rf', model2), ('lr', model3)], voting='soft')



In [103]:
# Reshape the input data
X_train_reshaped = X_train.reshape(X_train.shape[0], -1)
X_test_reshaped = X_test.reshape(X_test.shape[0], -1)

In [104]:
# Fit the ensemble model
ensemble_model.fit(X_train_reshaped, y_train)


C:\Users\bdiko\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


VotingClassifier(estimators=[('svm', SVC(probability=True)),
                             ('rf', RandomForestClassifier()),
                             ('lr', LogisticRegression())],
                 voting='soft')

In [105]:
# Evaluate the ensemble model
accuracy = ensemble_model.score(X_test_reshaped, y_test)
print(f"Ensemble Model Accuracy: {accuracy:.4f}")

Ensemble Model Accuracy: 0.7273


It looks like the ensemble model with the reshaped input data has achieved an accuracy of 0.7273. This indicates an improvement over the previous model.

Ensemble learning can often help improve the model's performance by leveraging the strengths of multiple models. In this case, combining different classifiers using a voting scheme has resulted in better accuracy.



## cross-validation and grid search to tune the hyperparameters of your ensemble model

In [106]:
from sklearn.model_selection import GridSearchCV

In [107]:
# Define the parameter grid for hyperparameter tuning
param_grid = {
    'svm__C': [0.1, 1, 10],
    'rf__n_estimators': [100, 200, 300],
    'lr__C': [0.1, 1, 10]
}

In [108]:
# Create the GridSearchCV object
grid_search = GridSearchCV(estimator=ensemble_model, param_grid=param_grid, cv=5)

In [109]:
# Fit the grid search to the training data
grid_search.fit(X_train_reshaped, y_train)

C:\Users\bdiko\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\bdiko\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\bdiko\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/st

GridSearchCV(cv=5,
             estimator=VotingClassifier(estimators=[('svm',
                                                     SVC(probability=True)),
                                                    ('rf',
                                                     RandomForestClassifier()),
                                                    ('lr',
                                                     LogisticRegression())],
                                        voting='soft'),
             param_grid={'lr__C': [0.1, 1, 10],
                         'rf__n_estimators': [100, 200, 300],
                         'svm__C': [0.1, 1, 10]})

In [110]:
# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)


Best Parameters: {'lr__C': 0.1, 'rf__n_estimators': 100, 'svm__C': 0.1}
Best Score: 0.6666666666666666


In [111]:
# Evaluate the ensemble model with the best parameters
accuracy = grid_search.score(X_test_reshaped, y_test)
print("Ensemble Model Accuracy:", accuracy)

Ensemble Model Accuracy: 0.7272727272727273


Accuracy is near the same using ensemble model.

## Lets try Transfer learning. 
- It is a powerful technique that allows you to leverage pre-trained models to improve the performance of your specific task. We applied transfer learning using a pre-trained modelVGG16, ResNet50 and InceptionV3.

In [206]:
from keras.applications import VGG16, ResNet50, InceptionV3
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.optimizers import Adam
from keras.utils import to_categorical


In [207]:
input_shape = (224, 224, 3)

## VGG16

In [208]:
# Choose the pre-trained model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
# base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
# base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=input_shape)

In [209]:
# Freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False


In [210]:
# Create a new model and add the pre-trained base model
model = Sequential()
model.add(base_model)

In [214]:
# Add your own classification layers on top
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(2, activation='softmax'))

In [215]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [216]:
# Train the model
model.fit(train_generator, epochs=10, validation_data=test_generator)

Epoch 1/10


3/3 [==============================] - 11s 4s/step - loss: 1.3890 - accuracy: 1.0000 - val_loss: 0.7201 - val_accuracy: 0.3333
Epoch 2/10
3/3 [==============================] - 11s 4s/step - loss: 1.3916 - accuracy: 1.0000 - val_loss: 0.7200 - val_accuracy: 0.3333
Epoch 3/10
3/3 [==============================] - 11s 4s/step - loss: 1.3915 - accuracy: 1.0000 - val_loss: 0.7198 - val_accuracy: 0.3333
Epoch 4/10
3/3 [==============================] - 12s 4s/step - loss: 1.3915 - accuracy: 1.0000 - val_loss: 0.7195 - val_accuracy: 0.3333
Epoch 5/10
3/3 [==============================] - 12s 4s/step - loss: 1.3914 - accuracy: 1.0000 - val_loss: 0.7193 - val_accuracy: 0.3333
Epoch 6/10
3/3 [==============================] - 13s 5s/step - loss: 1.3913 - accuracy: 1.0000 - val_loss: 0.7191 - val_accuracy: 0.3333
Epoch 7/10
3/3 [==============================] - 12s 4s/step - loss: 1.3912 - accuracy: 1.0000 - val_loss: 0.7189 - val_accuracy: 0.3333
Epoch 8/10
3/3 [=============================

In [217]:
# Evaluate the model on the test data
loss, accuracy = model.evaluate(test_generator, verbose=1)

# Print the evaluation results
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)


1/1 [==============================] - 2s 2s/step - loss: 0.7184 - accuracy: 0.3333
Test Loss: 0.7183578014373779
Test Accuracy: 0.3333333432674408


## ResNet50

In [218]:
# Choose the pre-trained model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)


94765736/94765736 [==============================] - 18s 0us/step


In [219]:
# Freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False


In [220]:
# Create a new model and add the pre-trained base model
model = Sequential()
model.add(base_model)

In [221]:
# Add your own classification layers on top
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(2, activation='softmax'))

In [222]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [223]:
# Train the model
model.fit(train_generator, epochs=10, validation_data=test_generator)

Epoch 1/10
3/3 [==============================] - 11s 3s/step - loss: 3.7806 - accuracy: 0.3263 - val_loss: 0.7049 - val_accuracy: 0.2917
Epoch 2/10
3/3 [==============================] - 7s 3s/step - loss: 2.2072 - accuracy: 0.9579 - val_loss: 1.4393 - val_accuracy: 0.3333
Epoch 3/10
3/3 [==============================] - 7s 3s/step - loss: 1.7377 - accuracy: 0.6737 - val_loss: 0.6770 - val_accuracy: 0.6667
Epoch 4/10
3/3 [==============================] - 8s 2s/step - loss: 1.7669 - accuracy: 0.0000e+00 - val_loss: 0.6470 - val_accuracy: 0.6667
Epoch 5/10
3/3 [==============================] - 8s 3s/step - loss: 1.4632 - accuracy: 0.6632 - val_loss: 0.9962 - val_accuracy: 0.3333
Epoch 6/10
3/3 [==============================] - 8s 3s/step - loss: 1.5798 - accuracy: 1.0000 - val_loss: 0.7319 - val_accuracy: 0.3333
Epoch 7/10
3/3 [==============================] - 8s 3s/step - loss: 1.4142 - accuracy: 0.3368 - val_loss: 0.6337 - val_accuracy: 0.6667
Epoch 8/10
3/3 [====================

In [224]:
# Evaluate the model on the test data
loss, accuracy = model.evaluate(test_generator, verbose=1)

# Print the evaluation results
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

1/1 [==============================] - 2s 2s/step - loss: 0.6804 - accuracy: 0.7083
Test Loss: 0.680443286895752
Test Accuracy: 0.7083333134651184


## InceptionV3

In [226]:
# Choose the pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=input_shape)

87910968/87910968 [==============================] - 16s 0us/step


In [227]:
# Freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False


In [228]:
# Create a new model and add the pre-trained base model
model = Sequential()
model.add(base_model)

In [229]:
# Add your own classification layers on top
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(2, activation='softmax'))

In [230]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [231]:
# Train the model
model.fit(train_generator, epochs=10, validation_data=test_generator)

Epoch 1/10
3/3 [==============================] - 9s 2s/step - loss: 2.9167 - accuracy: 0.4526 - val_loss: 1.1683 - val_accuracy: 0.5000
Epoch 2/10
3/3 [==============================] - 5s 2s/step - loss: 2.1997 - accuracy: 0.6632 - val_loss: 0.6445 - val_accuracy: 0.5833
Epoch 3/10
3/3 [==============================] - 5s 2s/step - loss: 1.9716 - accuracy: 0.5263 - val_loss: 0.9650 - val_accuracy: 0.5417
Epoch 4/10
3/3 [==============================] - 5s 2s/step - loss: 1.6226 - accuracy: 0.2947 - val_loss: 1.0364 - val_accuracy: 0.5417
Epoch 5/10
3/3 [==============================] - 5s 2s/step - loss: 1.6758 - accuracy: 0.8526 - val_loss: 0.7724 - val_accuracy: 0.6667
Epoch 6/10
3/3 [==============================] - 5s 2s/step - loss: 1.6441 - accuracy: 0.3579 - val_loss: 1.3412 - val_accuracy: 0.2917
Epoch 7/10
3/3 [==============================] - 5s 2s/step - loss: 1.7272 - accuracy: 0.6105 - val_loss: 0.8437 - val_accuracy: 0.6250
Epoch 8/10
3/3 [=========================

In [232]:
# Evaluate the model on the test data
loss, accuracy = model.evaluate(test_generator, verbose=1)

# Print the evaluation results
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)


1/1 [==============================] - 1s 982ms/step - loss: 0.7465 - accuracy: 0.5833
Test Loss: 0.7464776039123535
Test Accuracy: 0.5833333134651184


# Conclusion

### Ensemble Model Accuracy of 0.7273 is the highest one suggesting that this is the best model to be used for Rice Leaf Disease detection

Ensemble modeling is a technique that combines multiple individual models to make predictions. It has been widely used in machine learning to improve prediction accuracy and reduce model bias. Although the individual models may have slightly lower accuracy compared to the ensemble model, the ensemble approach can often yield better overall performance.

Here are a few reasons why ensemble modeling can be beneficial:

Improved Generalization: Ensemble models can reduce the risk of overfitting, which occurs when a model learns too much from the training data and performs poorly on unseen data. By combining multiple models that have been trained on different subsets of the data or using different algorithms, ensemble models can capture a broader range of patterns and make more robust predictions on unseen data.

Increased Stability: Ensemble models tend to be more stable than individual models. Even if some of the individual models in the ensemble make incorrect predictions, the combined predictions of all the models can still provide accurate results. This stability is especially useful when dealing with noisy or uncertain data.

Reduction of Bias: Different models may have different biases due to variations in their architectures, training data, or algorithms. By combining models with diverse biases, ensemble models can mitigate the individual biases and provide a more balanced prediction.

Error Correction: Ensemble models can correct errors made by individual models. If one model misclassifies a sample, other models in the ensemble can potentially correct that mistake, leading to more accurate predictions overall.

Better Performance on Unseen Data: Ensemble models often exhibit better performance on unseen data compared to individual models. By leveraging the collective knowledge and predictions of multiple models, ensemble models can achieve higher accuracy, robustness, and generalization capability.
